In [1]:
import asyncio
import os
from collections.abc import Sequence
from typing import Any

from dotenv import load_dotenv
from agent_framework import AgentSession, BaseHistoryProvider, Message
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity.aio import AzureCliCredential

In [2]:
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

Project Endpoint:  https://jdramkumar32-1674-resource.services.ai.azure.com/api/projects/jdramkumar32-1674
Model:  gpt-4o


In [3]:
class CustomHistoryProvider(BaseHistoryProvider):
    def __init__(self) -> None:
        super().__init__("custom-history")
        self._storage: dict[str, list[Message]] = {}

    async def get_messages(
        self, session_id: str | None, *, state: dict[str, Any] | None = None, **kwargs: Any
    ) -> list[Message]:
        key = session_id or "default"
        return list(self._storage.get(key, []))

    async def save_messages(
        self,
        session_id: str | None,
        messages: Sequence[Message],
        *,
        state: dict[str, Any] | None = None,
        **kwargs: Any,
    ) -> None:
        key = session_id or "default"

        if key not in self._storage:
            self._storage[key] = []
        self._storage[key].extend(messages)

In [4]:

credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=project_endpoint,
    deployment_name=model,
    credential=credential,
)

agent = client.as_agent(
    name="CustomBot",
    instructions="You are a helpful assistant that remembers our conversation.",
    context_providers=[CustomHistoryProvider()],
)

session = agent.create_session()

In [5]:
query = "Hello! My name is Alice and I love pizza."

print(f"User: {query}")
print(f"Agent: {await agent.run(query, session=session)}\n")

User: Hello! My name is Alice and I love pizza.
Agent: Hi Alice! It's great to meet you. Pizza is fantastic—do you have a favorite topping or style?



In [6]:
serialized_session = session.to_dict()

print(f"Serialized session: {serialized_session}\n")

resumed_session = AgentSession.from_dict(serialized_session)

Serialized session: {'type': 'session', 'session_id': 'e80ad333-97b9-4ca1-a904-1e40ed69ffc3', 'service_session_id': 'resp_04a631cd2431a7760069983360e67081948433404486d79e29', 'state': {}}



In [7]:
query = "What do you remember about me?"

agent2 = client.as_agent(
    name="CustomBot",
    instructions="You are a helpful assistant that remembers our conversation.",
    context_providers=[CustomHistoryProvider()],
)

print(f"User: {query}")
print(f"Agent: {await agent2.run(query, session=resumed_session)}\n")

User: What do you remember about me?
Agent: You mentioned that your name is Alice and you love pizza. Do you have any favorite toppings or styles?

